In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import make_scorer, mean_absolute_error as MAE

from Model import Machinery, pnl_score
from ChangePointDetection import ChangePointDetector

In [ ]:
rates = pd.read_csv("./data/input_rates.csv", index_col=0).values
rates = dict(rates)

date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
df = pd.read_excel('data/Project 2_2023.xlsx', sheet_name='Data', 
                    parse_dates=['Date'], date_parser=date_parser)
df = df.set_index('Date')
df.index.name = 'Date'

train_dates, test_dates = df[:'2021-01-01'].index, df['2021-01-01':'2021-03-31'].index
income, outcome = df["Income"], df["Outcome"]
target = (df["Income"] - df["Outcome"]).shift(-1)[:-1]

test_scores = []
pnl_scorer = make_scorer(pnl_score, greater_is_better=True, rates=rates)

change_point_detector = ChangePointDetector()

machine = Machinery(score=pnl_score, scorer=pnl_scorer, k_features=5)
machine.finetune(income[train_dates], outcome[train_dates], target[train_dates])

for date in test_dates:
    force_finetune = False
    for series in [income, outcome]:
        last_chp_date = change_point_detector.detect_changepoint(series[:date])
        if last_chp_date:
            if (date - last_chp_date).days < machine.finetune_every:
                force_finetune = True
                break
    if force_finetune:
        machine.finetune_count = machine.finetune_every
        machine.finetune(income[:date][:-1], outcome[:date][:-1], target[:date][:-1])

    prediction = machine.predict(income[:date], outcome[:date], horizon=1)

    score = pnl_score(target[date], prediction)
    mae_error = MAE(target[date], prediction)
    test_scores.append((date, mae_error, score))

    machine.finetune(income[:date], outcome[:date], target[:date])
    machine.calibrate_model(income[:date], outcome[:date], target[:date])


to_print = [f"{date.strftime('%Y-%m-%d')} {error} {score}" for date, error, score in test_scores]
with open("test_errors.txt", "w") as f:
    f.write("\n".join(to_print))